In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc = SparkContext("local", "SparkRddTuple")

22/05/11 02:03:47 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.174.129 instead (on interface ens33)
22/05/11 02:03:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/11 02:03:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/11 02:03:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/11 02:03:49 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/05/11 02:03:49 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/05/11 02:03:49 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
2

In [3]:
rdd = sc.textFile("hdfs://localhost:9000/ml-latest-small/movies.csv")


In [4]:
result = rdd.count()
print(result)

9743


In [5]:
# Answer to #1.1

#print(rdd.glom().collect())
print(rdd.take(5))

['movieId,title,genres', '1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy', '2,Jumanji (1995),Adventure|Children|Fantasy', '3,Grumpier Old Men (1995),Comedy|Romance', '4,Waiting to Exhale (1995),Comedy|Drama|Romance']


In [6]:
print(rdd.count())

9743


In [7]:
print(rdd.take(5))

['movieId,title,genres', '1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy', '2,Jumanji (1995),Adventure|Children|Fantasy', '3,Grumpier Old Men (1995),Comedy|Romance', '4,Waiting to Exhale (1995),Comedy|Drama|Romance']


In [8]:
headerLine = rdd.first()
rddList = rdd.filter (lambda line: line != headerLine)
print(rddList.take(5))


['1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy', '2,Jumanji (1995),Adventure|Children|Fantasy', '3,Grumpier Old Men (1995),Comedy|Romance', '4,Waiting to Exhale (1995),Comedy|Drama|Romance', '5,Father of the Bride Part II (1995),Comedy']


In [10]:
wordListRdd = rddList.map (lambda line: line.split(","))

wordListRdd.take(5)


[['1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'],
 ['2', 'Jumanji (1995)', 'Adventure|Children|Fantasy'],
 ['3', 'Grumpier Old Men (1995)', 'Comedy|Romance'],
 ['4', 'Waiting to Exhale (1995)', 'Comedy|Drama|Romance'],
 ['5', 'Father of the Bride Part II (1995)', 'Comedy']]

In [18]:
wordRdd = wordListRdd.flatMap(lambda elements: elements)
filteredRDD = wordRdd.filter(lambda x: x.startswith('Adventure'))

filteredRDD.count() # Answer to 1.2


658

In [21]:
romanceFilterRDD = wordRdd.filter(lambda x: "Romance" in x)
romanceFilterRDD.count() # Answer to 1.3

1603

In [22]:
comedyFilterRDD = wordRdd.filter(lambda x: "Comedy" in x)
comedyFilterRDD.count() # Answer to 1.2

3768

In [23]:
ratingRdd = sc.textFile("hdfs://localhost:9000/ml-latest-small/ratings.csv")


In [25]:
print(ratingRdd.take(10)) # Answer to 1.4

['userId,movieId,rating,timestamp', '1,1,4.0,964982703', '1,3,4.0,964981247', '1,6,4.0,964982224', '1,47,5.0,964983815', '1,50,5.0,964982931', '1,70,3.0,964982400', '1,101,5.0,964980868', '1,110,4.0,964982176', '1,151,5.0,964984041']


In [52]:
ratingListRdd = ratingRdd.map (lambda line: line.split(","))
print(ratingListRdd.take(5))


[['userId', 'movieId', 'rating', 'timestamp'], ['1', '1', '4.0', '964982703'], ['1', '3', '4.0', '964981247'], ['1', '6', '4.0', '964982224'], ['1', '47', '5.0', '964983815']]


In [93]:
ratingFilterRDD = ratingListRdd.filter(lambda x: x[2] >= '1.0')

print(ratingFilterRDD.take(20)) # Answer to 1.4

[['userId', 'movieId', 'rating', 'timestamp'], ['1', '1', '4.0', '964982703'], ['1', '3', '4.0', '964981247'], ['1', '6', '4.0', '964982224'], ['1', '47', '5.0', '964983815'], ['1', '50', '5.0', '964982931'], ['1', '70', '3.0', '964982400'], ['1', '101', '5.0', '964980868'], ['1', '110', '4.0', '964982176'], ['1', '151', '5.0', '964984041'], ['1', '157', '5.0', '964984100'], ['1', '163', '5.0', '964983650'], ['1', '216', '5.0', '964981208'], ['1', '223', '3.0', '964980985'], ['1', '231', '5.0', '964981179'], ['1', '235', '4.0', '964980908'], ['1', '260', '5.0', '964981680'], ['1', '296', '3.0', '964982967'], ['1', '316', '3.0', '964982310'], ['1', '333', '5.0', '964981179']]


In [103]:
ratingRdd2 = ratingFilterRDD.map (lambda line: tuple(line))
print(ratingRdd2.take(5))


[('userId', 'movieId', 'rating', 'timestamp'), ('1', '1', '4.0', '964982703'), ('1', '3', '4.0', '964981247'), ('1', '6', '4.0', '964982224'), ('1', '47', '5.0', '964983815')]


In [118]:
headerLine = ratingRdd2.first()
rddContent = ratingRdd2.filter (lambda line: line != headerLine)

ratingRdd3 = rddContent.map(lambda n: ((n[1], float(n[2]))))
print(ratingRdd3.take(10))
                                
#result = ratingRdd3.countByKey() 
#print(result.take(5))


[('1', 4.0), ('3', 4.0), ('6', 4.0), ('47', 5.0), ('50', 5.0), ('70', 3.0), ('101', 5.0), ('110', 4.0), ('151', 5.0), ('157', 5.0)]


In [120]:
result = ratingRdd3.reduceByKey(lambda acc, value: acc + value)
result.take(5)  # Answer to 1.5

[('1', 842.5), ('3', 169.0), ('6', 402.5), ('47', 806.0), ('50', 864.5)]